# Paste this on 'empirical_3'

## "Where exactly is the most dense area?"
Required : dat

In [ ]:
medals = pd.DataFrame(columns=['Gold', 'Gzip', 'Grho', 'Silver', 'Szip', 'Srho', 'Bronze', 'Bzip', 'Brho'], index=years)
for y,d in zip(years, dat):
    new_row = {}
    superhot = d.zbp.sort_values(by='rho', ascending=False).iloc[:3]
    new_row['Gold'] = superhot.iloc[0].GEO_TTL
    new_row['Gzip'] = superhot.iloc[0].ZIP
    new_row['Grho'] = superhot.iloc[0].rho
    new_row['Silver'] = superhot.iloc[1].GEO_TTL
    new_row['Szip'] = superhot.iloc[1].ZIP
    new_row['Srho'] = superhot.iloc[2].rho
    new_row['Bronze'] = superhot.iloc[2].GEO_TTL
    new_row['Bzip'] = superhot.iloc[2].ZIP
    new_row['Brho'] = superhot.iloc[2].rho
    medals.loc[y] = new_row
medals[['Gold', 'Silver', 'Bronze']]

## Comparing for various $\alpha$ values
Required : ykp

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(18,4))

for ax, df in zip(axes, [ykp_5, ykp_10, ykp_20]):
    g = df.groupby(['GEO_TTL'])
    X = g['k'].apply(lambda v: round(np.median(v))).to_numpy()
    X_D = g['k'].apply(lambda v: max(v)-min(v)).to_numpy()
    Y = g['P'].apply(np.median).to_numpy()
    Y_D = g['P'].apply(lambda v: max(v)-min(v)).to_numpy()

    kp = pd.DataFrame({'k':X, 'P':Y})
    h = kp.groupby(['k'])
    X_avg = list(h.indices)
    Y_avg = h['P'].mean()

    f = lambda x,a,b : a*(x**b)
    pfit, cov = curve_fit(f, X_avg[1:-1], Y_avg[1:-1])
    # Xfit = np.linspace(max(X_avg[1:-1]), min(X_avg[1:-1]), 10)
    Xfit = X_avg.copy()
    Yfit = f(Xfit, *pfit)

    std = np.sqrt(np.diag(cov))
    delta = pfit[1]
    R_sq = np.corrcoef(np.log(Y_avg[1:-1]), np.log(Yfit[1:-1]))[0,1]**2
    
    ax.set_xscale('log'); ax.set_yscale('log')
    ax.set_xlabel('k', fontsize='xx-large'); ax.set_ylabel('P', fontsize='xx-large')

    ax.scatter(X, Y, alpha = 0.3)
    ax.scatter(X_avg, Y_avg)
    ax.plot(Xfit[1:-1], Yfit[1:-1], ls='--', color='orange', alpha=0.8)

    antext = 'Slope = {:.2f} ± {:.3f} \nβ = {:.2f}, R² = {:.2f}'.format(delta, std[1], 1/delta, R_sq)
    ax.annotate(antext, (0.45, 0.05), xycoords='axes fraction', fontsize=13)
    print(std[1])
plt.show()

## Why don't I keep all the years?
Required : ykp

In [ ]:
X = ykp['k'].to_numpy()
Y = ykp['P'].to_numpy()

kp = pd.DataFrame({'k':X, 'P':Y})
h = kp.groupby(['k'])
X_avg = list(h.indices)
Y_avg = h['P'].mean()

fitfunc = lambda x,a,b : a*(x**b)
pfit, cov = curve_fit(fitfunc, X_avg[1:-1], Y_avg[1:-1])
Xfit = X_avg.copy()
Yfit = fitfunc(Xfit, *pfit)

Pstar = pfit[0]
delta = pfit[1]
std = np.sqrt(np.diag(cov))
R_sq = np.corrcoef(np.log(Y_avg[1:-1]), np.log(Yfit[1:-1]))[0,1]**2

print('pfit :', pfit)

In [ ]:
fig, ax = plt.subplots()
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlabel('k', fontsize='xx-large'); ax.set_ylabel('P', fontsize='xx-large')

ax.scatter(X, Y, alpha = 0.3)
ax.scatter(X_avg, Y_avg)
ax.plot(Xfit[1:-1], Yfit[1:-1], ls='--', color='orange', alpha=0.8)

antext = 'Slope = {:.2f} ± {:.2f} \nβ = {:.2f}, R² = {:.2f}'.format(delta, std[1], 1/delta, R_sq)
ax.annotate(antext, (0.45, 0.05), xycoords='axes fraction', fontsize=17)

plt.show()

## What the heck does KS-Test have to do with all this?
Required : ykp, result_KS (processing included below)

In [ ]:
from scipy.stats import ks_2samp
import itertools

In [1]:
ykp_sort = ykp.sort_values(by='GEO_TTL') # α for k threshold = 5
g_ykp = g = ykp.groupby(['GEO_TTL'])

count = lambda v: len(v)
geo_len = ykp.groupby(['GEO_TTL'])['k'].apply(count) 

lencut= 2
geo_len = geo_len[geo_len > lencut]

# k vs. <P> back from k-P ... k = 1~25
k_ref = g_ykp['k'].apply(lambda v: round(np.median(v)))
P_ref1 = Y_avg
P_ref2= g_ykp['P'].apply(np.median)

# Time Counter
tflag = time(); N = len(geo_len)
tdiv = 20; tarr = [round(T*N/tdiv) for T in range(1,tdiv+1)]

# Process
result_KS = []
for t, (thisgeo, thislen) in enumerate(geo_len.to_dict().items()):
    result_KS.append(pd.DataFrame(columns=['GEO_TTL', 'KS', 'p', 'ratio']))
    ratio = P_ref2[thisgeo] / P_ref1[k_ref[thisgeo]]
    
    df_ref = ykp_sort[ykp_sort['GEO_TTL'] == thisgeo]
    arr_sample = []
    for row in df_ref.iloc:
        d_search = [d for d in dat if d.year==row['Y']]
        if len(d_search) != 1: print('cannot find *d* by year')
        zbp = d_search[0].zbp
        df = zbp[zbp['GEO_TTL'] == thisgeo]
        sample_y = df['rho'].sort_values(ascending=False, ignore_index=True)
        arr_sample.append(sample_y)
    for sam1, sam2 in itertools.combinations(arr_sample,2):
        ks, p = ks_2samp(sam1, sam2)
        thisresult[0].append(ks)
        thisresult[1].append(p)
        new_row = {'GEO_TTL':thisgeo, 'KS':ks, 'p':p, 'ratio':ratio}
        result_KS[-1] = result_KS[-1].append(new_row, ignore_index=True)
    if t == tarr[0]:
        print(t, '/', N, '...', time()-tflag, 'sec')
        tarr.pop(0); tflag=time()
result_KS = pd.concat(tuple(result_KS), ignore_index=True)

NameError: name 'ykp' is not defined

In [ ]:
fig, ax = plt.subplots()
ax.set_ylabel("$P_k/<P_k>$", fontsize='xx-large')
ax.set_yscale('log')

ratios = result_KS['ratio'].to_numpy()
xtest = np.arange(len(ratios))
# ax.scatter(xtest, np.log(ratios)/np.log(10), s=10, alpha=0.2)
# ax.scatter(xtest, ratios, s=10, alpha=0.2)

bins=[-2.5, -2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5, 3]
ax.hist(np.log10(ratios), bins=bins, rwidth=0.5, align='left', alpha=0.7)
# ax.hist(ratios, bins=bins)

ax.axvline(x=0, color='tab:red', zorder = 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlabel('KS statistics', fontsize='xx-large'); ax.set_ylabel('$p_{KS}$', fontsize='xx-large')

X = result_KS['KS']
Y = result_KS['p']
ax.scatter(X, Y, s=10, alpha = 0.2, color='gray')

plt.show()

In [ ]:
# c1 = np.array([0, 0.263, 0.874]) # Deep Blue
c1 = np.array([0, 0.556, 0.458]) # Dark Green
c2 = np.array([0.863, 0, 0.215]) # Crimson

a, b = (c1-c2)/2, (c1+c2)/2; trimmer = 2
palette = lambda r: (a*r + b*abs(r)) / trimmer # r = log ratio

# Time Counter
tflag = time(); N = len(result_KS)
tdiv = 10; tarr = [round(T*N/tdiv) for T in range(1,tdiv+1)]

color_result_KS = []
for i, row in enumerate(result_KS.iloc):
    color = palette(np.log10(row['ratio']))
    color_result_KS.append(color)
    if i == tarr[0]:
        print(i, '/', N, '...', time()-tflag, 'sec')
        tarr.pop(0); tflag=time()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlabel('KS statistics', fontsize='xx-large'); ax.set_ylabel('$p_{KS}$', fontsize='xx-large')

X = result_KS['KS']
Y = result_KS['p']

tflag = time()
ax.scatter(X, Y, s=30, alpha = 0.2, color=color_result_KS)
print(time() - tflag)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlabel('<KS statistics>', fontsize='xx-large'); ax.set_ylabel('$<p_{KS}>$', fontsize='xx-large')

X = result_KS['KS']
Y = result_KS['p']

for X,Y in result:
    x,y = np.mean(X),np.mean(Y)
#     dx,dy = np.std(X),np.std(Y)
#     r = np.sqrt(dx)
    ax.scatter(x, y, alpha = 0.2, color='k')
    
plt.show()